In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.2.2
CUDA available: False


In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Define the Dataset class
class ECGDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = torch.FloatTensor(signals)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.signals)
    
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# Define the model (simplified architecture)
class SimpleECGNet(nn.Module):
    def __init__(self):
        super(SimpleECGNet, self).__init__()
        
        # First conv block
        self.conv1 = nn.Sequential(
            nn.Conv1d(12, 32, kernel_size=50, stride=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        # Second conv block
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=7),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        # Adaptive pooling to handle variable input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool1d(50)
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(64 * 50, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 3)  # 3 classes: Normal, AF, MI
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model = model.to(device)
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for signals, labels in train_loader:
            signals, labels = signals.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for signals, labels in val_loader:
                signals, labels = signals.to(device), labels.to(device)
                outputs = model(signals)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        # Print epoch statistics
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss/len(train_loader):.4f}')
        print(f'Val Loss: {val_loss/len(val_loader):.4f}')
        print(f'Val Accuracy: {100.*correct/total:.2f}%\n')

# Main execution
def main():
    # Check if CUDA is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Generate dummy data for demonstration
    # In practice, load your real ECG data here
    num_samples = 1000
    X = np.random.randn(num_samples, 12, 5000)  # 1000 samples, 12 leads, 5000 timepoints
    y = np.random.randint(0, 3, num_samples)    # Random labels (0: Normal, 1: AF, 2: MI)
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create data loaders
    train_dataset = ECGDataset(X_train, y_train)
    val_dataset = ECGDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # Initialize model, loss function, and optimizer
    model = SimpleECGNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device)
    
    return model

if __name__ == "__main__":
    model = main()


Using device: cpu
Epoch 1/10:
Train Loss: 1.2983
Val Loss: 1.0995
Val Accuracy: 38.00%

Epoch 2/10:
Train Loss: 1.1086
Val Loss: 1.0990
Val Accuracy: 38.00%

Epoch 3/10:
Train Loss: 1.0986
Val Loss: 1.0998
Val Accuracy: 35.50%

Epoch 4/10:
Train Loss: 1.1053
Val Loss: 1.0968
Val Accuracy: 35.50%

Epoch 5/10:
Train Loss: 1.0924
Val Loss: 1.0970
Val Accuracy: 35.50%

Epoch 6/10:
Train Loss: 1.0928
Val Loss: 1.0983
Val Accuracy: 35.50%

Epoch 7/10:
Train Loss: 1.0873
Val Loss: 1.0998
Val Accuracy: 30.00%

Epoch 8/10:
Train Loss: 1.0743
Val Loss: 1.0980
Val Accuracy: 35.50%

Epoch 9/10:
Train Loss: 1.0676
Val Loss: 1.1003
Val Accuracy: 35.50%

Epoch 10/10:
Train Loss: 1.0413
Val Loss: 1.1070
Val Accuracy: 35.50%

